## Import & config

In [207]:
import os
import sys
from glob import glob
import requests
import random
import numpy as np
import pandas as pd
import cv2

import torch
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm
from time import time
from torch.optim import *
import timm

import albumentations as A
from albumentations import *
from albumentations.pytorch import ToTensorV2

import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
# timm.list_models(pretrained=True)

In [189]:
### Configurations
train_folder_dir = 'G:/내 드라이브/Colab Notebooks/boost_camp_mask/level1_imageclassification_cv-level1-cv-06/T4064/dataset/train'
eval_folder_dir = 'G:/내 드라이브/Colab Notebooks/boost_camp_mask/level1_imageclassification_cv-level1-cv-06/T4064/dataset/eval'

train_imgs_dir = f'{train_folder_dir}/images'
train_labels_path = f'{train_folder_dir}/train.csv'
eval_imgs_dir = f'{eval_folder_dir}/images'
eval_labels_path = f'{eval_folder_dir}/info.csv'

cfg = {
    'model_name' : 'resnet34',
    'epochs' : 25,
    'lr' : 1e-4,
    'seed':42,
    'num_classes':18
    }


## Data Processing

### dataset

In [47]:
df = pd.read_csv(train_labels_path)
df

,id,gender,race,age,path
0,000001,female,Asian,45,000001_female_Asian_45
1,000002,female,Asian,52,000002_female_Asian_52
2,000004,male,Asian,54,000004_male_Asian_54
3,000005,female,Asian,58,000005_female_Asian_58
4,000006,female,Asian,59,000006_female_Asian_59
...,...,...,...,...,...
2695,006954,male,Asian,19,006954_male_Asian_19
2696,006955,male,Asian,19,006955_male_Asian_19
2697,006956,male,Asian,19,006956_male_Asian_19
2698,006957,male,Asian,20,006957_male_Asian_20


In [83]:
## csv 파일을 기반으로 img_path에 해당하는 class를 label_codes, img_paths 리스트에 저장하는 코드블럭

df = pd.read_csv(train_labels_path)

def get_label(mask_tag, gender, age):
    gender = gender.lower()
    label = 0
    if mask_tag == 'Wear' and gender == 'male' and age<30: # 0:
        label = 0
    elif mask_tag == 'Wear' and gender == 'male' and 30<=age<60: # 1
        label = 1
    elif mask_tag == 'Wear' and gender == 'male' and age>=60: # 2
        label = 2
    elif mask_tag == 'Wear' and gender == 'female' and age<30: # 3
        label = 3
    elif mask_tag == 'Wear' and gender == 'female' and 30<=age<60: # 4
        label = 4
    elif mask_tag == 'Wear' and gender == 'female' and age>=60: # 5
        label = 5
    elif mask_tag == 'Incorrect' and gender == 'male' and age<30: # 6
        label = 6
    elif mask_tag == 'Incorrect' and gender == 'male' and 30<=age<60: # 7
        label = 7
    elif mask_tag == 'Incorrect' and gender == 'male' and age>=60: # 8
        label = 8
    elif mask_tag == 'Incorrect' and gender == 'female' and age<30: # 9
        label = 9
    elif mask_tag == 'Incorrect' and gender == 'female' and 30<=age<60: # 10
        label = 10
    elif mask_tag == 'Incorrect' and gender == 'female' and age>=60: # 11
        label = 11
    elif mask_tag == 'Not Wear' and gender == 'male' and age<30: # 12
        label = 12
    elif mask_tag == 'Not Wear' and gender == 'male' and 30<=age<60: # 13
        label = 13
    elif mask_tag == 'Not Wear' and gender == 'male' and age>=60: # 14
        label = 14
    elif mask_tag == 'Not Wear' and gender == 'female' and age<30: # 15
        label = 15
    elif mask_tag == 'Not Wear' and gender == 'female' and 30<=age<60: # 16
        label = 16
    elif mask_tag == 'Not Wear' and gender == 'female' and age>=60: # 17
        label = 17
    else:
        raise ValueError
    return label

label_codes = []
img_paths = []

for gender, age, folder_name in zip(df['gender'], df['age'], df['path']):
    folder_path = os.path.join(train_imgs_dir, folder_name)
    files = os.listdir(folder_path)
    for file_name in files:
        img_path = os.path.join(folder_path,file_name)
        if 'incorrect' in file_name:
            label = get_label('Incorrect', gender, age)
        elif 'mask' in file_name:
            label = get_label('Wear', gender, age)
        elif 'normal' in file_name:
            label = get_label('Not Wear', gender, age)
        else:
            raise ValueError
        img_paths.append(img_path)
        label_codes.append(label)

In [129]:
for path, code in zip(img_paths, label_codes):
    print(path, code)
    break

G:/내 드라이브/Colab Notebooks/boost_camp_mask/level1_imageclassification_cv-level1-cv-06/T4064/dataset/train/images\000001_female_Asian_45\incorrect_mask.jpg 10


In [128]:
## 나이 클래스별 분포를 확인
counts = pd.DataFrame(label_codes).value_counts().sort_index()
young = counts[0]+counts[3]+counts[6]+counts[9]+counts[12]+counts[15]
middle = counts[1]+counts[4]+counts[7]+counts[10]+counts[13]+counts[16]
old = counts[2]+counts[5]+counts[8]+counts[11]+counts[14]+counts[17]
print(young, middle, old)

8967 8589 1344


In [149]:
## 데이터 스플릿
train_paths, valid_paths, train_labels, valid_labels = train_test_split(img_paths, label_codes,
                                                                        train_size = 0.7,
                                                                        shuffle = True,
                                                                        random_state = hyper_params['seed'],
                                                                        stratify=label_codes)

In [150]:
train_csv = pd.DataFrame(list(zip(train_paths, train_labels)), columns=['img_path', 'class'])

In [151]:
valid_csv = pd.DataFrame(list(zip(valid_paths, valid_labels)), columns=['img_path', 'class'])

In [152]:
print(len(train_csv), len(valid_csv))

13230 5670


In [153]:
train_csv

,img_path,class
0,G:/내 드라이브/Colab Notebooks/boost_camp_mask/leve...,0
1,G:/내 드라이브/Colab Notebooks/boost_camp_mask/leve...,16
2,G:/내 드라이브/Colab Notebooks/boost_camp_mask/leve...,4
3,G:/내 드라이브/Colab Notebooks/boost_camp_mask/leve...,4
4,G:/내 드라이브/Colab Notebooks/boost_camp_mask/leve...,4
...,...,...
13225,G:/내 드라이브/Colab Notebooks/boost_camp_mask/leve...,4
13226,G:/내 드라이브/Colab Notebooks/boost_camp_mask/leve...,4
13227,G:/내 드라이브/Colab Notebooks/boost_camp_mask/leve...,7
13228,G:/내 드라이브/Colab Notebooks/boost_camp_mask/leve...,3


In [193]:
## csv파일을 받아서 리턴하는 Custom Dataset Class

class MaskDataSet(Dataset):#train_labels_path
    def __init__(self, path_df, transforms=None):
        self.transforms = transforms
        self.path_df = path_df
        if transforms is None: # transform이 없으면 자동으로 3개의 transform이 적용됨.
            print('If transforms param is None automatically Resize(224), Normalize, ToTensorV2 Apply')
    
    def __getitem__(self, index):
        img_path = self.path_df['img_path'][index]
        img = np.array(Image.open(img_path))
        img_class = self.path_df['class'][index]
        if self.transforms is not None:
            img = self.transforms(image=img)["image"]
        else:
            temp_transforms = A.Compose([A.Resize(height=224, width=224), A.Normalize(), ToTensorV2()])
            img = temp_transforms(image=img)["image"]
        return img, img_class

    def __len__(self):
        return len(self.path_df)

In [194]:
train_dset = MaskDataSet(train_csv)
img, label = train_dset[1]
img.size(), label

If transforms param is None automatically Resize(224), Normalize, ToTensorV2 Apply


(torch.Size([3, 224, 224]), 16)

In [199]:
train_transforms = A.Compose([A.Resize(height=224, width=224),
                              A.HorizontalFlip(p=0.5),
                              A.RandomBrightnessContrast(p=0.5),
                              A.GaussianBlur(p=0.5),
                              A.GridDistortion(p=0.5),
                              A.Rotate(limit=30, p=0.5),
                              A.Normalize(mean=(0.56019358,0.52410121,0.501457),
                                          std=(0.23318603,0.24300033,0.24567522)),
                              ToTensorV2()])

train_dataset = MaskDataSet(train_csv, train_transforms)
valid_dataset = MaskDataSet(valid_csv)

If transforms param is None automatically Resize(224), Normalize, ToTensorV2 Apply


In [200]:
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=64, shuffle=True)

In [201]:
i, l = train_dataset[1]

In [202]:
i.size()

torch.Size([3, 224, 224])

## Modeling

In [192]:
def get_pretrain_model(config):
    model_name = config['model_name']
    model = timm.create_model(model_name=config['model_name'], pretrained=True, num_classes=config['num_classes'])
    return model
pretrained_model = get_pretrain_model(cfg)
pretrained_model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act1): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (drop_block): Identity()
      (act1): ReLU(inplace=True)
      (aa): Identity()
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act2): ReLU(inplace=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, m

In [212]:
## 테스트
batch_images, batch_labels = next(iter(train_loader))
torch.argmax(pretrained_model(batch_images), dim=-1)

tensor([ 8, 10, 15, 10, 12,  0, 13,  7,  8, 13,  0,  5, 10, 15,  8,  8,  0, 12,
        12,  9, 15, 11, 13,  0,  8,  0,  8,  5, 16,  3,  7, 14,  3,  9,  3,  2,
        10, 13,  7, 10, 15,  7,  3,  8,  8, 12, 10,  8,  9,  3, 16,  8,  6,  8,
         5,  8, 10,  3,  0, 10, 10,  3,  8,  7])

## Training

In [ ]:
## metric code

In [184]:
def train_and_valid(config, model, train_loader, valid_loader):
    return None

In [ ]:
## plot result code